In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from src.MLP import MLP
# from src.validate import validate


In [2]:
import numpy as np
import pandas as pd

# Load the provided dataset
file_path = 'data/data_100000.csv'
data = pd.read_csv(file_path)

In [3]:
from sklearn.metrics import r2_score, mean_squared_error
from tqdm.auto import tqdm


def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Convert dataset to PyTorch tensors
X = torch.tensor(data[['l2', 'l3', 'l4', 'EE_x', 'EE_y']].values, dtype=torch.float32)
y = torch.tensor(data[['d_max', 'gr_min']].values, dtype=torch.float32)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = torch.tensor(scaler.fit_transform(X_train), dtype=torch.float32)
X_test = torch.tensor(scaler.transform(X_test), dtype=torch.float32)

# Create data loaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define the model
model = MLP(in_channels=5, dim=50, out_channels=2)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 100


# Early stopping parameters
best_test_rmse = float('inf')
patience, patience_counter = 5, 0

for epoch in tqdm(range(num_epochs), desc="Training Progress", unit="epoch"):
    model.train()  # Set the model to training mode
    train_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    model.eval()  # Set the model to evaluation mode for testing
    with torch.no_grad():
        # Predict on training set
        train_preds = model(X_train).numpy()
        train_r2 = r2_score(y_train.numpy(), train_preds)
        train_rmse = rmse(y_train.numpy(), train_preds)

        # Predict on testing set
        test_preds = model(X_test).numpy()
        test_r2 = r2_score(y_test.numpy(), test_preds)
        test_rmse = rmse(y_test.numpy(), test_preds)

    # Print progress at the end of each epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Avg Loss: {train_loss/len(train_loader):.4f}, '
          f'Train R2: {train_r2:.4f}, Train RMSE: {train_rmse:.4f}, '
          f'Test R2: {test_r2:.4f}, Test RMSE: {test_rmse:.4f}')

    # Early stopping logic
    if test_rmse < best_test_rmse:
        best_test_rmse = test_rmse
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break

c:\Users\Jihoon\miniconda3\envs\torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Training Progress:   1%|          | 1/100 [00:03<05:12,  3.16s/epoch]

Epoch 1/100, Avg Loss: 0.4137, Train R2: 0.9797, Train RMSE: 0.2972, Test R2: 0.9785, Test RMSE: 0.3051


Training Progress:   2%|▏         | 2/100 [00:06<05:11,  3.18s/epoch]

Epoch 2/100, Avg Loss: 0.0798, Train R2: 0.9868, Train RMSE: 0.2312, Test R2: 0.9857, Test RMSE: 0.2411


Training Progress:   3%|▎         | 3/100 [00:09<05:10,  3.20s/epoch]

...

Epoch 34/100, Avg Loss: 0.0138, Train R2: 0.9967, Train RMSE: 0.1125, Test R2: 0.9964, Test RMSE: 0.1182


Training Progress:  35%|███▌      | 35/100 [02:07<03:57,  3.65s/epoch]

Epoch 35/100, Avg Loss: 0.0139, Train R2: 0.9968, Train RMSE: 0.1075, Test R2: 0.9967, Test RMSE: 0.1093


Training Progress:  35%|███▌      | 35/100 [02:10<04:03,  3.74s/epoch]

Epoch 36/100, Avg Loss: 0.0140, Train R2: 0.9967, Train RMSE: 0.1170, Test R2: 0.9963, Test RMSE: 0.1242
Early stopping triggered.


In [19]:
import random
from deap import base, creator, tools, algorithms

TARGET_D_MAX = 1.0 
TARGET_GR_MIN = 2.0

BOUNDS = [(0.050001, 0.9497), (0.083289, 2.0), (0.093218, 2.907), (-0.49998, 2.5), (-1.5, 1.5)]
# BOUNDS = [(-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5)]

# Surrogate model prediction function
def surrogate_model_predict(individual):
    input_tensor = torch.tensor([individual], dtype=torch.float32)
    scaled_input = torch.tensor(scaler.transform(input_tensor), dtype=torch.float32)
    model.eval()
    with torch.no_grad():
        predicted_targets = model(scaled_input)
    d_max_pred, gr_min_pred = predicted_targets.numpy()[0]
    return d_max_pred, gr_min_pred

# Fitness function
def evaluate(individual):
    # d_max_pred, gr_min_pred = surrogate_model_predict(individual)
    d_max_pred, gr_min_pred = surrogate_model_predict(individual)
    # TARGET_D_MAX = 0.5  # Replace with your actual target
    # TARGET_GR_MIN = 0.5  # Replace with your actual target
    return abs(d_max_pred - TARGET_D_MAX), abs(gr_min_pred - TARGET_GR_MIN)

# Set up DEAP
creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, 0, 1)
toolbox.register("individual", tools.initIterate, creator.Individual, lambda: [random.uniform(b[0], b[1]) for b in BOUNDS])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selNSGA2)

def checkBounds(min, max):
    def decorator(func):
        def wrapper(*args, **kwargs):
            offspring = func(*args, **kwargs)
            for child in offspring:
                for i in range(len(child)):
                    if child[i] > max[i]:
                        child[i] = max[i]
                    elif child[i] < min[i]:
                        child[i] = min[i]
            return offspring
        return wrapper
    return decorator

toolbox.decorate("mate", checkBounds([b[0] for b in BOUNDS], [b[1] for b in BOUNDS]))
toolbox.decorate("mutate", checkBounds([b[0] for b in BOUNDS], [b[1] for b in BOUNDS]))

# NSGA-II Algorithm
def main():
    NGEN = 50  # Number of generations
    MU = 100    # Population size
    LAMBDA = 200
    CXPB = 0.7
    MUTPB = 0.2

    pop = toolbox.population(n=MU)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    algorithms.eaMuPlusLambda(pop, toolbox, mu=MU, lambda_=LAMBDA, cxpb=CXPB, mutpb=MUTPB, 
                             ngen=NGEN, stats=stats, halloffame=hof, verbose=True)

    return pop, stats, hof

if __name__ == "__main__":
    pop, stats, hof = main()

c:\Users\Jihoon\miniconda3\envs\torch\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Jihoon\miniconda3\envs\torch\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg    	std    	min      	max    
0  	100   	3.92119	3.62648	0.0233234	13.9503
1  	179   	0.991598	1.08333	0.00771701	5.71452
2  	179   	0.308785	0.231944	0.00771701	1.38438
3  	180   	0.200795	0.226375	0.000628114	1.06641
4  	179   	0.20169 	0.301665	0.000628114	2.3862 
5  	188   	0.131348	0.213798	0.000628114	1.06641
6  	176   	0.0856175	0.146118	0.000202298	0.799603
7  	179   	0.0718635	0.125091	0.000202298	0.704628
8  	178   	0.0645908	0.114943	0.000202298	0.614002
9  	180   	0.0704572	0.133923	0.000202298	0.614002
10 	180   	0.0636913	0.121877	0.000202298	0.548784
11 	178   	0.0641021	0.108312	0.000202298	0.548784
12 	181   	0.0496651	0.0972456	0.000202298	0.548784
13 	180   	0.0403555	0.0763738	0.000202298	0.357693
14 	178   	0.0480341	0.0921784	0.000202298	0.357693
15 	181   	0.0690589	0.113851 	0.000202298	0.357693
16 	179   	0.113259 	0.145791 	0.000202298	0.357693
17 	182   	0.147064 	0.15784  	0.000202298	0.318011
18 	179   	0.140873 	0.157081 	0.000202298	0.31801

In [20]:
import os
import pandas as pd

# Function to run NSGA-II for a given set of target conditions
def run_nsga_ii(target_d_max, target_gr_min, filename):
    # Set the target values in the evaluate function
    def evaluate(individual):
        # d_max_pred, gr_min_pred = surrogate_model_predict(individual)
        # d_max_pred, gr_min_pred = validate(individual) # with ground truth
        d_max_pred, gr_min_pred = surrogate_model_predict(individual) # with surrogate model
        return abs(d_max_pred - target_d_max), abs(gr_min_pred - target_gr_min)

    toolbox.register("evaluate", evaluate)

    # Run the NSGA-II algorithm
    pop, stats, hof = main()

    # Convert the hall of fame individuals to a DataFrame
    pop_df = pd.DataFrame([ind for ind in pop], columns=['l2', 'l3', 'l4', 'EE_x', 'EE_y'])

    # Save to CSV
    pop_df.to_csv(filename, index=False)

# Ensure the output directory exists
output_dir = "./synthesis_100k/NSGA-II_v2"
os.makedirs(output_dir, exist_ok=True)

# Target conditions sets
target_conditions = [
    (0.4, 5.0),
    (1.0, 2.0),
    (2.0, 1.5),
    (2.5, 1.0)
]

# target_conditions = [
#     (2.5, 10.0)
# ]

# Run NSGA-II for each set of target conditions and save results
for i, (target_d_max, target_gr_min) in enumerate(target_conditions, start=1):
    filename = os.path.join(output_dir, f"nsga_ii_results_set_{i}_{target_d_max}_{target_gr_min}.csv")
    run_nsga_ii(target_d_max, target_gr_min, filename)
    print(f"Results for set {i} saved to {filename}")


gen	nevals	avg    	std    	min       	max    
0  	100   	5.56649	4.01312	0.00480331	19.7194
1  	178   	2.09404	1.85069	0.00432282	11.9883
2  	182   	1.10058	1.78172	0.00432282	11.9883
3  	177   	0.294625	0.350914	0.00221289	1.53734
4  	178   	0.141709	0.195743	0.000219887	1.17147
5  	186   	0.112507	0.172427	0.000219887	1.17147
6  	177   	0.108457	0.118065	0.000219887	0.577624
7  	180   	0.110925	0.122889	0.000219887	0.577624
8  	177   	0.115846	0.14524 	0.000219887	0.583227
9  	183   	0.0817945	0.098956	2.84076e-05	0.27188 
10 	179   	0.0619448	0.0847678	2.84076e-05	0.237076
11 	175   	0.0235054	0.0456666	2.84076e-05	0.237076
12 	184   	0.02516  	0.0479361	2.84076e-05	0.237076
13 	178   	0.0188189	0.0339919	2.84076e-05	0.237076
14 	173   	0.0177922	0.0304254	2.84076e-05	0.237076
15 	183   	0.0202935	0.0373963	2.84076e-05	0.237076
16 	180   	0.0216156	0.0408208	2.84076e-05	0.237076
17 	176   	0.0178578	0.0301537	2.84076e-05	0.237076
18 	183   	0.0174453	0.0315192	2.84076e-05	0.237076
1

In [9]:
pd.DataFrame(pop).describe()

,0,1,2,3,4
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.150852,4.420709,2.314502,-1.631098,-3.555529
std,0.032455,0.355032,0.110967,0.396963,0.404783
min,0.051599,1.340248,0.999701,-1.714867,-3.613034
25%,0.147655,4.470260,2.325664,-1.683739,-3.613034
50%,0.147655,4.470260,2.325664,-1.683739,-3.613034
75%,0.147655,4.470260,2.325664,-1.683739,-3.613034
max,0.457356,4.470260,2.404933,1.976494,0.316255


In [10]:
from src.validate import validate

for i in pop:
    d_max_pred, gr_min_pred = surrogate_model_predict(i)
    print(d_max_pred, gr_min_pred)

1.5530347 1.9995642
1.5530347 1.9995642
0.9999826 1.9533465
0.9999826 1.9533465
0.99892783 1.9909515
1.2482585 2.0004976
1.2482585 2.0004976
0.96006835 2.001602
1.1336087 2.0015666
1.1876624 2.001529
1.2482585 2.0004976
1.1876624 2.001529
1.1336087 2.0015666
1.0187949 1.9979122
0.9893091 1.9965439
0.99892783 1.9909515
0.9975649 2.0069537
0.9975649 2.0069537
0.96006835 2.001602
1.0028714 1.9937124
1.0028714 1.9937124
1.0097741 1.9947855
1.0097741 1.9947855
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
1.5530347 1.9995642
